## Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
import warnings;warnings.filterwarnings(action='ignore')

## Read data

In [2]:
train = pd.read_csv('../data/stayed_train.csv')
test = pd.read_csv('../data/stayed_test.csv')

In [3]:
print('학습데이터 수:', train.shape)
print('평가데이터 수:', test.shape)

학습데이터 수: (127741, 18)
평가데이터 수: (40148, 17)


In [4]:
'''
데이터 설명
- sessionID : 세션 ID
- userID : 사용자 ID
- TARGET : 세션에서 발생한 총 조회수
- browser : 사용된 브라우저
- OS : 사용된 기기의 운영체제
- device : 사용된 기기
- new : 첫 방문 여부 (0: 첫 방문 아님, 1: 첫 방문)
- quality : 세션의 질 (거래 성사를 기준으로 측정된 값, 범위: 1~100)
- duration : 총 세션 시간 (단위: 초)
- bounced : 이탈 여부 (0: 이탈하지 않음, 1: 이탈함)
- transaction : 세션 내에서 발생의 거래의 수
- transaction_revenue : 총 거래 수익
- continent : 세션이 발생한 대륙
- subcontinent : 세션이 발생한 하위 대륙
- country : 세션이 발생한 국가
- traffic_source : 트래픽이 발생한 소스
- traffic_medium : 트래픽 소스의 매체
- keyword : 트래픽 소스의 키워드, 일반적으로 traffic_medium이 organic, cpc인 경우에 설정
- referral_path : traffic_medium이 referral인 경우 설정되는 경로
'''
display(train.head())

,sessionID,userID,TARGET,browser,OS,device,new,quality,duration,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,17.0,Chrome,Macintosh,desktop,0,45.0,839.0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
1,SESSION_000001,USER_000001,3.0,Chrome,Windows,desktop,1,1.0,39.0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
2,SESSION_000007,USER_000007,5.0,Chrome,Macintosh,desktop,1,1.0,64.0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
3,SESSION_000008,USER_000008,5.0,Firefox,Linux,desktop,1,1.0,60.0,0.0,0.0,Americas,South America,Brazil,youtube.com,referral,NaN,Category5_Path_0002
4,SESSION_000009,USER_000009,3.0,Chrome,Macintosh,desktop,1,2.0,579.0,0.0,0.0,Americas,Northern America,United States,google,cpc,Category1_Keyword_000,NaN


### 결측치
keyword, referral_path에는 traffic_medium값에 따른 결측치가 존재한다.<br>

####  Traffic Medium
- keyword 변수값을 결정하는 traffic_medium은 웹사이트로 유입되는 트래픽의 구체적인 유형이나 채널을 의미한다.
- 'organic'(무료): 사용자가 검색 엔진에서 특정 키워드를 **검색**하고 결과 페이지에서 나타난 링크로 접속
- 'affiliate': **다른 회사**의 제품이나 서비스를 홍보하고, 그 결과로 발생하는 판매나 클릭에 대해 수수료를 받음
- 'referral': 블로그, 파트너 사이드 등의 다른 웹사이트의 링크를 통해 접속
- 'cpc'(클릭당 비용 발생): **광고** 캠페인(예: Google AdWords)을 통해 웹사이트에 유입
- 'cpm'(천 회 노출당 비용 발생): 디스플레이나 비디오 **광고**에 노출되어 접속
- '(none)'은 traffic_source가 (direct)인  경우이다.<br>
  direct: 사용자가 웹 브라우저에 URL을 직접 입력하거나, 즐겨찾기를 사용해 웹사이트에 직접 접속하는 경우
- '(not set)': 결측치로 봐야함

In [5]:
train.isna().sum()

sessionID                  0
userID                     0
TARGET                     0
browser                    0
OS                         0
device                     0
new                        0
quality                    0
duration                   0
transaction                0
transaction_revenue        0
continent                  0
subcontinent               0
country                    0
traffic_source             0
traffic_medium             0
keyword                70399
referral_path          79591
dtype: int64

In [6]:
# keyword, referral_path의 값을 결정하는 traffic_medium의 빈도를 살펴보니 결측치로 판단할 수 있는 (none), (not set)이 있다.
train.traffic_medium.value_counts()

organic      55247
(none)       35686
referral     30738
cpc           3416
affiliate     2477
cpm            174
(not set)        3
Name: traffic_medium, dtype: int64

In [7]:
# (none)은 직접 트래픽이라 트래픽 매체가 없는 것이고 (not set)은 결측치로 봐야한다.
print('(none)의 트래픽 소스:', train.query('traffic_medium == "(none)"').traffic_source.unique())
print('(not set)의 트래픽 소스:', train.query('traffic_medium == "(not set)"').traffic_source.unique())

(none)의 트래픽 소스: ['(direct)']
(not set)의 트래픽 소스: ['Partners' 'google']


In [8]:
# (not set)으로 결측치가 표현된 다른 변수가 있는지 확인한 결과 OS, continent, subcontinent, country에서도 나타난다.
train.applymap(lambda x: x=="(not set)").sum()

sessionID                 0
userID                    0
TARGET                    0
browser                   0
OS                     1044
device                    0
new                       0
quality                   0
duration                  0
transaction               0
transaction_revenue       0
continent               148
subcontinent            148
country                 148
traffic_source            0
traffic_medium            3
keyword                   0
referral_path             0
dtype: int64

In [9]:
# traffic_medium 중 광고로 접속한 cpc, cpm은 ad(광고)로 변경한다.
train['traffic_medium'] = train['traffic_medium'].replace('cpc','ad').replace('cpm','ad')
test['traffic_medium'] = test['traffic_medium'].replace('cpc','ad').replace('cpm','ad')

# (not set)은 결측치로 학습데이터 내 트래픽 소스의 최빈값으로 채운다.
mode = train.groupby('traffic_source')['traffic_medium'].agg(lambda x: x.mode()[0])
ease = train.query('traffic_medium == "(not set)"')['traffic_source'].map(mode)
train.loc[ease.index, 'traffic_medium'] = ease.values

ease = test.query('traffic_medium == "(not set)"')['traffic_source'].map(mode)
test.loc[ease.index, 'traffic_medium'] = ease.values

### 변수 정리

In [4]:
# browser 내 device 정보가 섞여 범주값이 다양하다. 학습데이터를 기반으로 재구성한다.
# 결측치로 생각하는 (not set)도 포함한다.
unclear_browser = ["Mozilla Compatible Agent", "MRCHROME", "+Simple Browser",
                   'SeaMonkey', 'osee2unifiedRelease', 'YE', 'Browser', 'starmaker', '(not set)']

train['browser'] = train['browser'].apply(lambda x: 'Safari' if 'Safari' in x else
                                                     ('Opera' if 'Opera' in x else
                                                     ('Amazon' if 'Amazon' in x else
                                                     ('BROWSER' if x in unclear_browser else x))))
test['browser'] = test['browser'].apply(lambda x: 'Safari' if 'Safari' in x else
                                                     ('Opera' if 'Opera' in x else
                                                     ('Amazon' if 'Amazon' in x else
                                                     ('BROWSER' if x in unclear_browser else x))))

In [5]:
# 같은 이유로 OS도 재구성한다.
train['OS'] = train['OS'].apply(lambda x: 'Windows' if 'Windows' in x else
                                           ('Nintendo' if 'Nintendo' in x else                                
                                           ('OS' if x in ['OS/2','(not set)'] else x)))
test['OS'] = test['OS'].apply(lambda x: 'Windows' if 'Windows' in x else
                                           ('Nintendo' if 'Nintendo' in x else                                
                                           ('OS' if x in ['OS/2','(not set)'] else x)))

In [6]:
# traffic_source를 지역 도메인을 제외해 재구성한다.
# google엔 analytics, mail, groups, adwords & ads, optimize, docs, earth 등이 섞여있다.
train['traffic_source'] = train.traffic_source.str.replace('.com|.net|.org', '')\
                          .apply(lambda x: 'yahoo' if '.yahoo' in x else
                                            ('facebook' if '.facebook' in x else
                                            ('youtube' if '.youtube' in x else
                                            ('pinterest' if '.pinterest' in x else
                                            ('edu' if '.edu' in x else
                                            ('ask' if '.ask' in x else
                                            ('reddit' if '.reddit' in x else
                                            ('google' if 'google' in x else x))))))))

test['traffic_source'] = test.traffic_source.str.replace('.com|.net|.org', '')\
                         .apply(lambda x: 'yahoo' if '.yahoo' in x else
                                            ('facebook' if '.facebook' in x else
                                            ('youtube' if '.youtube' in x else
                                            ('pinterest' if '.pinterest' in x else
                                            ('edu' if '.edu' in x else
                                            ('ask' if '.ask' in x else
                                            ('reddit' if '.reddit' in x else
                                            ('google' if 'google' in x else x))))))))

### 변수 생성

In [10]:
train[['keyword1','keyword2']] = train['keyword'].str.split('_', expand=True)[[0,2]]
test[['keyword1','keyword2']] = test['keyword'].str.split('_', expand=True)[[0,2]]

train.fillna("(not set)", inplace=True)
test.fillna("(not set)", inplace=True)

In [11]:
train[['referral_path1','referral_path2']] = train['referral_path'].str.split('_', expand=True)[[0,2]]
test[['referral_path1','referral_path2']] = test['referral_path'].str.split('_', expand=True)[[0,2]]

train.fillna("(not set)", inplace=True)
test.fillna("(not set)", inplace=True)

In [12]:
# 불필요한 열 제거
train.drop(['keyword', 'referral_path'], axis=1, inplace=True)
test.drop(['keyword', 'referral_path'], axis=1, inplace=True)

### 이상치 제거

In [13]:
outlier = [train.transaction.sort_values().index[-1], train.TARGET.sort_values().index[-1]]
print('제거 전 데이터 크기:', train.shape)
train.drop(outlier, inplace=True)
print('제거 후 데이터 크기:', train.shape)

제거 전 데이터 크기: (127741, 20)
제거 후 데이터 크기: (127739, 20)


In [14]:
display(train)

,sessionID,userID,TARGET,browser,OS,device,new,quality,duration,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword1,keyword2,referral_path1,referral_path2
0,SESSION_000000,USER_000000,17.0,Chrome,Macintosh,desktop,0,45.0,839.0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,(not set),(not set),(not set)
1,SESSION_000001,USER_000001,3.0,Chrome,Windows,desktop,1,1.0,39.0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,(not set),(not set),(not set)
2,SESSION_000007,USER_000007,5.0,Chrome,Macintosh,desktop,1,1.0,64.0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,(not set),(not set),(not set)
3,SESSION_000008,USER_000008,5.0,Firefox,Linux,desktop,1,1.0,60.0,0.0,0.0,Americas,South America,Brazil,youtube.com,referral,(not set),(not set),Category5,0002
4,SESSION_000009,USER_000009,3.0,Chrome,Macintosh,desktop,1,2.0,579.0,0.0,0.0,Americas,Northern America,United States,google,ad,Category1,000,(not set),(not set)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127736,SESSION_252281,USER_206018,6.0,Chrome,Macintosh,desktop,1,1.0,99.0,0.0,0.0,Asia,Eastern Asia,Hong Kong,google,organic,Category8,(not set),(not set),(not set)
127737,SESSION_252282,USER_206019,3.0,Chrome,Macintosh,desktop,1,2.0,1423.0,0.0,0.0,Americas,Northern America,United States,google,organic,(not set),(not set),(not set),(not set)
127738,SESSION_252286,USER_164240,5.0,Chrome,Macintosh,desktop,0,2.0,69.0,0.0,0.0,Americas,Northern America,United States,(direct),(none),(not set),(not set),Category1,(not set)
127739,SESSION_252287,USER_206023,1.0,Android Webview,Android,mobile,1,1.0,28.0,0.0,0.0,Africa,Northern Africa,Egypt,youtube.com,referral,(not set),(not set),Category2,0018


## Save data

In [15]:
train.to_csv('../data/stayed_train.csv', index=False)
test.to_csv('../data/stayed_test.csv', index=False)